PEDV的细胞内shape数据处理

1）设置不同大小的窗口

In [44]:
import numpy as np

genid = "MK584552.1"
genlen = 28044
win_size = 25
step_size = 1
stt=1
win=[]

while(stt+win_size <= int(genlen)):#得到所有窗口
    end=stt+win_size
    l=[genid,str(stt),str(end)]
    win.append(l)
    stt=stt+step_size
win_matrix = np.array(win)
win_matrix.shape

(28019, 3)

2）xml文件转shape反应性值

In [32]:
import xml.etree.ElementTree as ET
count=0
with open("D:/360MoveData/Users/dgwei/Desktop/PEDV-shape/rep2/MK584552.1.react.txt","w")as F:
    tree = ET.parse('D:/360MoveData/Users/dgwei/Desktop/PEDV-shape/rep2/MK584552.1.xml')
    root = tree.getroot()
    for line in root:
        a=root[0][0].text
        b=root[0][1].text
        for i in b.split(","):
            ii=i.strip()
            count+=1
            if ii=="NaN":
                ii=0
            F.write(str(count)+"\t"+str(ii)+"\n")

3）wig文件转shannon值

In [33]:
import pandas as pd

def fill_missing_shannon_values(file_path, output_file=None):
   # 读取CSV文件，假设没有标题行，以逗号分隔
   df_shannon = pd.read_table(file_path, header=None, names=['V1', 'V2'], sep='\t', dtype={'V1': int, 'V2': float})
   # 创建一个包含从1到28044的序列的新DataFrame
   df_shannon_new = pd.DataFrame(range(1, 28045), columns=['V1'])
   # 将新创建的DataFrame与Shannon值DataFrame根据'V1'列合并， outer join确保所有行都包含在结果中
   df_shannon_newest = pd.merge(df_shannon_new, df_shannon, on='V1', how='outer')
   # 将合并后的DataFrame中的NaN值替换为0
   df_shannon_newest.fillna(0, inplace=True)
   # 确保V2列是数值类型
   df_shannon_newest['V2'] = df_shannon_newest['V2'].astype(float)
   # 如果需要，将结果保存到新的CSV文件
   if output_file:
       df_shannon_newest.to_csv(output_file, index=False,sep='\t',header=None)
   return df_shannon_newest

file_path = "D:/360MoveData/Users/dgwei/Desktop/PEDV-shape/rep2/MK584552.1.wig"
output_file = "D:/360MoveData/Users/dgwei/Desktop/PEDV-shape/rep2/MK584552.1.shannon.txt"

df_completed = fill_missing_shannon_values(file_path, output_file)

4)SHAPE值和shannon值均一化

In [34]:
import numpy as np
val=[];spl_val=[]
with open("D:/360MoveData/Users/dgwei/Desktop/PEDV-shape/rep2/MK584552.1.win25react.txt","w")as ff:
    with open("D:/360MoveData/Users/dgwei/Desktop/PEDV-shape/rep2/MK584552.1.react.txt","r")as F:
        for line in F:
            val.append(float(line.split('\t')[1].rstrip()))
        val_array=np.array(val)
        val_median=np.median(val_array)
        print(val_median)
    num_rows, num_cols = win_matrix.shape
    for i in range(num_rows):
        start = win_matrix[i,1]
        end = win_matrix[i,2]
        spl_val.append(str(np.median(val_array[int(start)-1:int(end)])-val_median))
    ff.write(str('\n'.join(spl_val)))

0.232


In [35]:
val=[];spl_val=[]
with open("D:/360MoveData/Users/dgwei/Desktop/PEDV-shape/rep2/MK584552.1.win25shannon.txt","w")as ff:
    with open("D:/360MoveData/Users/dgwei/Desktop/PEDV-shape/rep2/MK584552.1.shannon.txt","r")as F:
        for line in F:
            val.append(float(line.split('\t')[1].rstrip()))
        val_array=np.array(val)
        val_median=np.median(val_array)
        print(val_median)
    num_rows, num_cols = win_matrix.shape
    for i in range(num_rows):
        start = win_matrix[i,1]
        end = win_matrix[i,2]
        spl_val.append(str(np.median(val_array[int(start)-1:int(end)])))
    ff.write(str('\n'.join(spl_val)))

0.0086


5)找出rep1和rep2中共同的wellfold区域

In [8]:
import os
import numpy as np
os.chdir("D:/360MoveData/Users/dgwei/Desktop/PEDV-shape/")
pos=0;b=[];c=[];a=[];s=[];val1=[];val2=[];val3=[];val4=[]
with open("D:/360MoveData/Users/dgwei/Desktop/PEDV-shape/MK584552.1.50.highrect.lowsh.txt",'w')as FF:
    with open("./rep1/MK584552.1.win25react.txt",'r')as f1,open("./rep1/MK584552.1.win25shannon.txt",'r') as f2,\
            open("./rep2/MK584552.1.win25react.txt",'r') as f3,open("./rep2/MK584552.1.win25shannon.txt",'r') as f4:
        for l1 in f1:
            l2=f2.readline();l3=f3.readline();l4=f4.readline()
            val1.append(float(l1.rstrip()));val2.append(float(l2.rstrip()))
            val3.append(float(l3.rstrip()));val4.append(float(l4.rstrip()))
        val_array1 = np.array(val1);val_array2 = np.array(val2)
        val_array3 = np.array(val3);val_array4 = np.array(val4)
        val_median1 = np.percentile(val_array1,50);val_median2 = np.percentile(val_array2,50)
        val_median3 = np.percentile(val_array3,50);val_median4 = np.percentile(val_array4,50)

        for ll1, ll2, ll3 ,ll4 in zip(val1, val2, val3, val4):
            pos += 1
            if float(ll1) > val_median1 and float(ll2) < val_median2:
                b.append(pos+12)
            if float(ll3) > val_median3 and float(ll4) < val_median4:
                c.append(pos+12)
        a = list (set(b).intersection(set(c)))

        for i in sorted(set(a)):
            if len(s) == 0 or s[-1] + 1 == i:
                s.append(i)  # 入栈
            else:
                if len(s) >= 2:
                    print(s)
                    FF.write(str(s[0]) + '\t' + str(s[-1]) + "\n")
                s = []  # 清空
                s.append(i)  # 入栈

[44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56]
[74, 75, 76, 77, 78]
[117, 118, 119, 120]
[124, 125]
[316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328]
[351, 352, 353, 354, 355, 356, 357, 358]
[371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381]
[413, 414, 415, 416, 417, 418, 419, 420, 421]
[427, 428, 429, 430, 431, 432]
[488, 489, 490, 491, 492, 493, 494]
[502, 503]
[508, 509, 510, 511, 512, 513, 514, 515, 516]
[518, 519, 520, 521, 522]
[549, 550, 551, 552, 553, 554]
[637, 638, 639, 640]
[696, 697, 698, 699, 700]
[708, 709, 710, 711, 712, 713, 714]
[1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028]
[1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261]
[1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290]
[1547, 1548]
[1550, 1551, 1552, 1553, 1554, 1555, 1556, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564, 1565, 1566, 1567, 1568, 1569, 1570, 1571, 1572, 1573, 1574, 15